<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/2023/ex11notebookB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2023 ex11notebookB

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo11.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2023

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

In [ ]:
# scikit-learn の判別分析のためのクラスを使えるように import
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

# 学習データとテストデータを分ける関数
from sklearn.model_selection import train_test_split

----
## もっと判別分析してみよう
----


---
### ワインの判別



前回とは別のデータで判別分析してみましょう．ここで扱うのは，[UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php) にある [Wine Data Set](https://archive.ics.uci.edu/ml/datasets/wine) です．
scikit-learn にこのデータを扱う関数 [sklearn.datasets.load_wine](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html) があるので，それを使います．



In [ ]:
# Wine Data Set
from sklearn.datasets import load_wine
data = load_wine(as_frame=True)
dfWine = data.frame
dfWine

これは，イタリアのある地方で作られた3種類（おそらくブドウの品種が異なる3種類）のワインに対して化学的な分析を行って得られた数値から成るデータです．
13個の数値から3クラスの判別を行います．

In [ ]:
# ペアプロット（実行に少し時間がかかります）
seaborn.pairplot(dfWine, hue='target', palette='tab10', corner=True)

上記のデータを np.array に加工します．

In [ ]:
# dfWine の target 列以外を X に，target 列を Y に
X = dfWine.drop(columns='target').to_numpy()
print(X[:5, :], X.shape)
Y = dfWine['target'].to_numpy(dtype=int)
print(Y, Y.shape)

#### 問題1

次のセルに，次のことを行うコードを書きなさい
1. `X` と `Y` のデータに対して線形判別分析を行い，各データのクラスを予測した結果を変数 `Yt` に代入する
1. `Y` と `Yt` を使っていくつのデータで正解したかを計算し，その数とデータの総数を表示する

#### 問題2

問題1と同じことを，二次判別分析でやろう．scikit-learn の [sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis.html) を使いましょう．

#### 問題3

上記のワインの判別の問題について，次の問の答えをメモしておきなさい．

1. データの次元数（変数の数）
1. 平均は何次元のものが何個？分散共分散行列は何行何列のものが何個？

クラス数とデータの次元数がわかれば 2. の答えはすぐ分かりますが，コードの出力を見て確認したい場合は，`LinearDiscriminantAnalysis(store_covariance=True)` や `QuadraticDiscriminantAnalysis(store_covariance=True)` のように `store_covariance` オプションを `True` にして，生成されるオブジェクトの `means_` や `covariance_` という変数の shape を調べたらよいでしょう．ただし，`covariance_` に複数の分散共分散行列が格納される場合，np.array のリストの形となるので，`len(hoge.covariance_)` が行列の個数，`hoge.covariance_[0].shape` がその中の先頭の行列の shape を表します．

---
### 学習とテスト

上記の実験では，与えられたデータを全て使って判別分析のパラメータを推定し，同じデータに対して予測結果の精度を調べています．この場合，正解を知っているデータで予測の精度を評価しているので，クラスが未知のデータに対しても同様の精度が得られるかどうか不確かです．

予測の精度をもっときちんと評価したい場合，データを二つに分けて，一方のデータのみを使ってパラメータを推定し，他方の（パラメータ推定に使わなかった方の）データで予測の精度を評価する，というやり方をします．このとき，前者のデータを **学習データ** (learning data)または **訓練データ** (training data) といい，後者のデータを **テストデータ** (test data) といいます．



以下で，Wine Data Set を学習データとテストデータに分けて判別分析する実験をやってみましょう．
学習データとテストデータをあらかじめ分けて提供するデータセットもありますが，Wine Data Set では分けられていませんので，適当に自分で分けることにします．

scikit-learn の [sklearn.model_selection.train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) 関数を使うと，与えられたデータを簡単に学習用とテスト用に分けることができます．
次のセルのコードを実行すると，`X` と `Y` に格納されている値のうち，ランダムに選んだ 6 割のものが `X_train` と `Y_train` に入り，残りが `X_test` と `Y_test` に入ります．

In [ ]:
# データのうち6割を学習データとし，残りをテストデータとする
#   デフォルトではランダムに選ばれるが，ここでは同じ条件で実験できるように random_state を指定
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.6, random_state=2)
print('### 学習データ')
print(X_train[:3, :], X_train.shape)
print(Y_train, Y_train.shape)
print()
print('### テストデータ')
print(X_test[:3, :], X_test.shape)
print(Y_test, Y_test.shape)
print()

次のセルを実行すると，学習データを使って線形判別分析の学習（パラメータの推定）を行い，判別を行います．

In [ ]:
model = LinearDiscriminantAnalysis(store_covariance=True)
model.fit(X_train, Y_train)
Yt = model.predict(X_train)
ncorrect = np.sum(Yt == Y_train)
print(f'学習データ: 正解数/データ数 = {ncorrect}/{len(X_train)}')

#### 問題4

テストデータを判別し正解数を出力するコードを書きなさい．

**注意**
線形判別のパラメータはすでに学習データ（ `X_train` と `Y_train` ）を使って推定されています（上記の `fit` がそれをやっている）．ここで `fit` を呼ぶことはないはず．


このデータセットの判別は簡単すぎて，学習データとテストデータを分けたことの影響はあまり実感できないかもしれません．そこは notebookC の方で...．

#### 問題5

問題2と同じことを，二次判別分析でやろう．scikit-learn の [sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis.html) を使いましょう．

このデータセットでは，線形判別と二次判別の違いも見えないかもしれません．そこは notebookC の方で...．

#### 問題4 & 5 の答え

以下のコードセルの `False` を `True` にして実行すると，答えの数字が print されます．


In [ ]:
himitsu = np.array([5858, 5892, 5857, 5893], dtype=int)
if False:
    hoge = 5963 - himitsu
    print('問題4')
    print(f'   学習データ: 正解数/データ数 = {hoge[0]}/{len(X_train)}')
    print(f'   テストデータ: 正解数/データ数 = {hoge[1]}/{len(X_test)}')
    print()
    print('問題5')
    print(f'   学習データ: 正解数/データ数 = {hoge[2]}/{len(X_train)}')
    print(f'   テストデータ: 正解数/データ数 = {hoge[3]}/{len(X_test)}')